# Analyzing PBMC scATAC-seq

In this notebook, we will analyze PBMC scATAC-seq data. 

Following [vignette](https://stuartlab.org/signac/articles/pbmc_vignette) and a [video](https://www.youtube.com/watch?v=yEKZJVjc5DY&t=1033s) by bioinfomagician.

## Preparation

Here are the data we will use. Data can be obtained by either using `wget` command or from the [vignette](https://stuartlab.org/signac/articles/pbmc_vignette).

``` 
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_singlecell.csv
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz
wget https://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz.tbi
```

We can save all the files in `'../data/atac'` after downloading.

In [1]:
list.files(path = '../data/atac')

[1] "atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5"
[2] "atac_v1_pbmc_10k_fragments.tsv.gz"          
[3] "atac_v1_pbmc_10k_fragments.tsv.gz.tbi"      
[4] "atac_v1_pbmc_10k_singlecell.csv"

Install packages if needed.

In [2]:
# remotes::install_github("stuart-lab/signac", ref="develop")
# install.packages("Matrix", type = "source")
# install.packages("irlba", type = "source")
# BiocManager::install("EnsDb.Hsapiens.v75")

In [3]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v75)
library(tidyverse)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading re

## Look at the data

This is what frag file looks like.

In [4]:
frag.file <- read.delim('../data/atac/atac_v1_pbmc_10k_fragments.tsv.gz', header = F, nrows = 10)
head(frag.file)

,V1,V2,V3,V4,V5
,<chr>,<int>,<int>,<chr>,<int>
1,chr1,10066,10279,TTAGCTTAGGAGAACA-1,2
2,chr1,10072,10279,TTAGCTTAGGAGAACA-1,2
3,chr1,10079,10316,ATATTCCTCTTGTACT-1,2
4,chr1,10084,10340,CGTACAAGTTACCCAA-1,1
5,chr1,10085,10271,TGTGACAGTACAACGG-1,1
6,chr1,10085,10339,CATGCCTTCTCTGACC-1,1


There are five columns in this data. Here is a quick overview on each column:

    - First column: Reference genome used to identify chromosome.
    - Second: Starting position of the fragment.
    - Third: End position of the fragment.
    - Fourth: Barcode fragment, which is used to identify sample. Same as `CB` tag in BAM file.
    - Fifth: Total number of reads associated with this fragment.
    
More information about fragment data can be found on https://support.10xgenomics.com/single-cell-atac/software/pipelines/latest/output/fragments.

Let's take a look at this now. If you get an error saying `Error in Read10X_h5("../data/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5"): Please install hdf5r to read HDF5 files`, install `libhdf5-dev` using 

`sudo apt-get install libhdf5-dev` (using terminal on debian/ubuntu)

and 

`install.packages('hdf5r')` on R.

You can also install it using github and devtool:
``` R
install.packages('devtools')
devtools::install_github("hhoeflin/hdf5r")
```

If you are using anaconda, you have to install `hdfr5` on conda using `mamba install hdf5` then install `hdf5r` on R.

In [5]:
install.packages('devtools')
devtools::install_github("hhoeflin/hdf5r")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Skipping install of 'hdf5r' from a github remote, the SHA1 (dc4774c0) has not changed since last install.
  Use `force = TRUE` to force installation



In [6]:
counts <- Read10X_h5('../data/atac/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5')
counts[1:10,1:10]

  [[ suppressing 10 column names 'AAACGAAAGAGCGAAA-1', 'AAACGAAAGAGTTTGA-1', 'AAACGAAAGCGAGCTA-1' ... ]]



10 x 10 sparse Matrix of class "dgCMatrix"
                                      
chr1:565107-565550 . . . . . . . . . .
chr1:569174-569639 . . . . . . . . . .
chr1:713460-714823 . . 2 8 . . 2 2 . .
chr1:752422-753038 . . . . . . . . . .
chr1:762106-763359 . . 4 2 . . . . 2 .
chr1:779589-780271 . . . . . . . . . .
chr1:793516-793741 . . . 1 . . . . . .
chr1:801120-801338 . . . . . . . . . .
chr1:804872-805761 . . . 4 . . . . . .
chr1:839520-841123 . . 2 2 . . . . . .

The first number is the range of open chromatin. Dots mean zeros. Numbers after the range represent `tn5` integration site in each cell that maps to the region. `tn5` is an enzyme that binds to open chromatin region. Learn more about `tn5`: https://www.youtube.com/watch?v=uuxpyhGNDsk. 

`chrom_assay` has many information in it.

In [7]:
chrom_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c(":", "-"),
  fragments = "../data/atac/atac_v1_pbmc_10k_fragments.tsv.gz",
  min.cells = 10,
  min.features = 200
)

str(chrom_assay)

Computing hash



Formal class 'ChromatinAssay' [package "Signac"] with 16 slots
  ..@ ranges            :Formal class 'GRanges' [package "GenomicRanges"] with 7 slots
  .. .. ..@ seqnames       :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. .. .. ..@ values         : Factor w/ 24 levels "chr1","chr2",..: 1 2 3 4 5 6 7 8 9 10 ...
  .. .. .. .. ..@ lengths        : int [1:24] 8555 6858 5499 3515 4389 5269 4263 3677 3540 4137 ...
  .. .. .. .. ..@ elementMetadata: NULL
  .. .. .. .. ..@ metadata       : list()
  .. .. ..@ ranges         :Formal class 'IRanges' [package "IRanges"] with 6 slots
  .. .. .. .. ..@ start          : int [1:87561] 565107 569174 713460 752422 762106 779589 793516 801120 804872 839520 ...
  .. .. .. .. ..@ width          : int [1:87561] 444 466 1364 617 1254 683 226 219 890 1604 ...
  .. .. .. .. ..@ NAMES          : NULL
  .. .. .. .. ..@ elementType    : chr "ANY"
  .. .. .. .. ..@ elementMetadata: NULL
  .. .. .. .. ..@ metadata       : list()
  .. .. ..@ strand

In [8]:
metadata <- read.csv(file = '../data/atac/atac_v1_pbmc_10k_singlecell.csv', header = T, row.names = 1)
View(metadata)

,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,cell_id,is__cell_barcode,TSS_fragments,DNase_sensitive_region_fragments,enhancer_region_fragments,promoter_region_fragments,on_target_fragments,blacklist_region_fragments,peak_region_fragments
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
NO_BARCODE,8335327,3922818,71559,746476,634014,22972,2937488,None,0,0,0,0,0,0,0,0
AAACGAAAGAAACGCC-1,3,1,0,1,0,0,1,None,0,1,1,1,0,1,0,1
AAACGAAAGAAAGCAG-1,14,1,0,4,2,0,7,None,0,0,2,0,0,2,0,0
AAACGAAAGAAAGGGT-1,7,1,0,1,0,0,5,None,0,2,2,0,1,2,0,1
AAACGAAAGAAATACC-1,9880,5380,79,106,1120,6,3189,None,0,386,1623,297,107,1758,16,262
AAACGAAAGAAATCTG-1,2,0,0,0,0,0,2,None,0,1,2,1,1,2,0,2
AAACGAAAGAAATGGG-1,4,2,0,1,0,0,1,None,0,0,1,1,0,1,0,1
AAACGAAAGAAATTCG-1,2,0,0,0,1,0,1,None,0,0,0,0,0,0,0,0
AAACGAAAGAACCATA-1,12,3,0,8,0,0,1,None,0,0,1,0,0,1,0,0


In [9]:
pbmc <- CreateSeuratObject(
  counts = chrom_assay,
  meta.data = metadata,
  assay = 'ATAC'
)

In [10]:
str(pbmc)

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 1
  .. ..$ ATAC:Formal class 'ChromatinAssay' [package "Signac"] with 16 slots
  .. .. .. ..@ ranges            :Formal class 'GRanges' [package "GenomicRanges"] with 7 slots
  .. .. .. .. .. ..@ seqnames       :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. .. .. .. .. .. ..@ values         : Factor w/ 24 levels "chr1","chr2",..: 1 2 3 4 5 6 7 8 9 10 ...
  .. .. .. .. .. .. .. ..@ lengths        : int [1:24] 8555 6858 5499 3515 4389 5269 4263 3677 3540 4137 ...
  .. .. .. .. .. .. .. ..@ elementMetadata: NULL
  .. .. .. .. .. .. .. ..@ metadata       : list()
  .. .. .. .. .. ..@ ranges         :Formal class 'IRanges' [package "IRanges"] with 6 slots
  .. .. .. .. .. .. .. ..@ start          : int [1:87561] 565107 569174 713460 752422 762106 779589 793516 801120 804872 839520 ...
  .. .. .. .. .. .. .. ..@ width          : int [1:87561] 444 466 1364 617 1254 683 226 219 890 1604 ...


We want to store annotations into pbmc Serat object.

In [11]:
pbmc@assays$ATAC@annotation

NULL

When we look at the `annotations`, we see that they are not compatible with 10x genomics pbmc data we have. We need to add `chr` in front of each seqnames column. This is because the data was mapped to `hg19`.

In [12]:
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v75)

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

In [13]:
annotations

GRanges object with 3072120 ranges and 5 metadata columns:
                  seqnames        ranges strand |           tx_id   gene_name
                     <Rle>     <IRanges>  <Rle> |     <character> <character>
  ENSE00001489430        X 192989-193061      + | ENST00000399012      PLCXD1
  ENSE00001536003        X 192991-193061      + | ENST00000484611      PLCXD1
  ENSE00002160563        X 193020-193061      + | ENST00000430923      PLCXD1
  ENSE00001750899        X 197722-197788      + | ENST00000445062      PLCXD1
  ENSE00001489388        X 197859-198351      + | ENST00000381657      PLCXD1
              ...      ...           ...    ... .             ...         ...
  ENST00000361739       MT     7586-8269      + | ENST00000361739      MT-CO2
  ENST00000361789       MT   14747-15887      + | ENST00000361789      MT-CYB
  ENST00000361851       MT     8366-8572      + | ENST00000361851     MT-ATP8
  ENST00000361899       MT     8527-9207      + | ENST00000361899     MT-ATP6
  ENS

Add the gene information to the object.

In [14]:
Annotation(pbmc) <- annotations
pbmc@assays$ATAC@annotation

GRanges object with 3072120 ranges and 5 metadata columns:
                  seqnames        ranges strand |           tx_id   gene_name
                     <Rle>     <IRanges>  <Rle> |     <character> <character>
  ENSE00001489430        X 192989-193061      + | ENST00000399012      PLCXD1
  ENSE00001536003        X 192991-193061      + | ENST00000484611      PLCXD1
  ENSE00002160563        X 193020-193061      + | ENST00000430923      PLCXD1
  ENSE00001750899        X 197722-197788      + | ENST00000445062      PLCXD1
  ENSE00001489388        X 197859-198351      + | ENST00000381657      PLCXD1
              ...      ...           ...    ... .             ...         ...
  ENST00000361739       MT     7586-8269      + | ENST00000361739      MT-CO2
  ENST00000361789       MT   14747-15887      + | ENST00000361789      MT-CYB
  ENST00000361851       MT     8366-8572      + | ENST00000361851     MT-ATP8
  ENST00000361899       MT     8527-9207      + | ENST00000361899     MT-ATP6
  ENS

## Compute Quality Control

Compute nucleosome signal score per cell

In [15]:
pbmc <- NucleosomeSignal(pbmc)

Compute TSS enrichment score per cell

In [ ]:
pbmc <- TSSEnrichment(object = pbmc, fast = FALSE)

Extracting TSS positions



Add blacklist ratio and fraction of reads in peaks

In [ ]:
pbmc$blacklist_ratio <- pbmc$blacklist_region_fragments / pbmc$peak_region_fragments
pbmc$pct_reads_in_peaks <- pbmc$peak_region_fragments / pbmc$passed_filters * 100

In [ ]:
View(pbmc@meta.data)

## Visualizing Quality Control

In [ ]:
colnames(pbmc@meta.data)

In [ ]:
a1 <- DensityScatter(pbmc, x = 'nCount_ATAC', y = 'TSS.enrichment', log_x = TRUE, quantiles = TRUE)
a2 <- DensityScatter(pbmc, x = 'nucleosome_signal', y = 'TSS.enrichment', log_x = TRUE, quantiles = TRUE)

a1 | a2

In [ ]:
VlnPlot(object = pbmc, 
        features = c('nCount_ATAC', 'nFeature_ATAC', 'TSS.enrichment', 'nucleosome_signal', 'blacklist_ratio', 'pct_reads_in_peaks'),
        pt.size = 0.1,
        ncol = 6)

## Filtering poor quality cells

In [ ]:
pbmc <- subset(x = pbmc,
               subset = nCount_ATAC > 3000 &
                 nCount_ATAC < 30000 &
                 pct_reads_in_peaks > 15 & 
                 blacklist_ratio < 0.05 &
                 nucleosome_signal < 4 &
                 TSS.enrichment > 3)

## Normalization and linear dimensional reduction

In [ ]:
pbmc <- RunTFIDF(pbmc) # normalization
pbmc <- FindTopFeatures(pbmc, min.cutoff = 'q0') # selecting top features
pbmc <- RunSVD(pbmc) # dimensionality reduction

DepthCor(pbmc)

## Non-linear dimensional reduction and Clustering

In [ ]:
pbmc <- RunUMAP(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindNeighbors(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindClusters(object = pbmc, algorithm = 3)

DimPlot(object = pbmc, label = TRUE) + NoLegend()